In [5]:
import pandas as pd
from tabulate import tabulate
from google.colab import drive
drive.mount('/content/drive')

input_file= '/content/drive/My Drive/fyp/final-preprocess/Reviews_SriLankan_destinations.csv'
try:
    df = pd.read_csv(input_file, encoding="utf-8")
except UnicodeDecodeError:
    df = pd.read_csv(input_file, encoding="ISO-8859-1")

# Function to clean text encoding artifacts
def clean_text(text):
    if isinstance(text, str):
        return (
            text.replace("Â’", "'")
            .replace("Â‘", "'")
            .replace("Â", "'")
            .replace("’", "'")
            .replace("‘", "'")
            .replace("Â", "")
            .replace("", "'")
        )
    return text

# Apply cleaning function using `.map()`
for col in df.select_dtypes(include=[object]).columns:
    df[col] = df[col].map(clean_text)

columns_to_save = ["Location_Name", "Located_City", "Location_Type", "User_Location", "User_Locale", "User_Contributions", "Rating", "Helpful_Votes", "Title", "Text"]
df_selected = df[columns_to_save].copy()

# Print number of unique location names
unique_location_count = df_selected['Location_Name'].nunique()
print(f"\nTotal unique locations: {unique_location_count}")



print(tabulate(location_counts, headers='keys', tablefmt='fancy_grid'))

# Print total number of rows
total_rows = len(df_selected)
print(f"\n Total number of rows in dataset: {total_rows}")
print(tabulate(df_selected.head(16), headers="keys", tablefmt="fancy_grid"))

output_file = "/content/drive/My Drive/fyp/final-preprocess/Reviews_SriLankan_destinations-encoded.csv"
df_selected.to_csv(output_file, index=False, encoding="utf-8")
print(f"Cleaned data saved to: {output_file}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Total unique locations: 76
╒════╤═════════════════════════════════════════════╤═════════╕
│    │ Location_Name                               │   count │
╞════╪═════════════════════════════════════════════╪═════════╡
│  0 │ Pinnawala Elephant Orphanage                │     369 │
├────┼─────────────────────────────────────────────┼─────────┤
│  1 │ Royal Botanical Gardens                     │     337 │
├────┼─────────────────────────────────────────────┼─────────┤
│  2 │ Polonnaruwa                                 │     333 │
├────┼─────────────────────────────────────────────┼─────────┤
│  3 │ Wilpattu National Park                      │     326 │
├────┼─────────────────────────────────────────────┼─────────┤
│  4 │ Dambulla Cave Temple                        │     321 │
├────┼─────────────────────────────────────────────┼─────────┤
│  5 │ Sigiriya Museum  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install contractions


In [ ]:
import pandas as pd
import re
from tabulate import tabulate
import contractions

def preprocess_text(text):
    if pd.isna(text):
        return text
    text = contractions.fix(text)
    # Replace "'s" with " is" and "'re" with " are" before removing special characters
    text = re.sub(r"'s\b", ' is', text)
    text = re.sub(r"'re\b", ' are', text)
    # Remove common emoticons (expand as needed)
    text = re.sub(r'(:\)|:\(|:D|:P|;\)|:-\)|:-\(|:-D|:-P|;-\))', '', text)
    # Remove special characters except alphanumeric, space, and period
    text = re.sub(r'[^\w\s\.]', '', text)
    # Replace multiple periods with a single period
    text = re.sub(r'\.+', '.', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Remove leading/trailing spaces
    text = text.strip()
    return text

df = pd.read_csv("/content/drive/My Drive/fyp/final-preprocess/Reviews_SriLankan_destinations-encoded.csv")

# Apply preprocessing
df['Text'] = df['Text'].apply(preprocess_text)

# Save cleaned data
df.to_csv("/content/drive/My Drive/fyp/final-preprocess/Reviews_SriLankan_destinations-preprocessed.csv", index=False)

# Show first 16 rows
print(tabulate(df.head(16), headers="keys", tablefmt="fancy_grid"))


╒════╤═════════════════╤════════════════╤═════════════════╤════════════════════════════════════╤═══════════════╤══════════════════════╤══════════╤═════════════════╤══════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

In [ ]:
import pandas as pd
import re
from tabulate import tabulate

df = pd.read_csv("/content/drive/My Drive/fyp/final-preprocess/Reviews_SriLankan_destinations-preprocessed.csv")

# Function to split review text into sentences
def split_into_sentences(text):
    if pd.isna(text):
        return []
    sentences = re.split(r'\.\s*', text)
    sentences = [s.strip() for s in sentences if s.strip()]

    return sentences

# Create the new column with list of sentences
df['review sentences'] = df['Text'].apply(split_into_sentences)

# Explode so each sentence gets its own row, keeping all original columns
df_exploded = df.explode('review sentences').reset_index(drop=True)
# Ensure each sentence is lowercase
df_exploded['review sentences'] = df_exploded['review sentences'].str.lower()

# Save to new CSV (all columns included)
df_exploded.to_csv("/content/drive/My Drive/fyp/final-preprocess/Reviews_SriLankan_destinations-sentences.csv", index=False)

# Show first 16 rows using tabulate
print(tabulate(df_exploded.head(36), headers="keys", tablefmt="fancy_grid"))


╒════╤═════════════════╤════════════════╤═════════════════╤═════════════════════════════╤═══════════════╤══════════════════════╤══════════╤═════════════════╤══════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

In [ ]:
%%writefile aspect_vb.py
aspect_vb = {
    'Attractions': [
        "attraction", "sightseeing", "landmark", "monument", "highlight", "must-see", "point of interest",
        "tourist spot", "viewpoint", "scenery", "scenic", "iconic place", "famous site", "place to visit",
        "sight", "wonder", "exhibit", "gallery", "museum", "amusement park", "theme park", "zoo", "aquarium",
        "botanical garden", "art gallery", "observatory", "planetarium", "park", "plaza", "square", "statue",
        "memorial", "tower", "bridge", "castle", "palace", "fortress", "ruin", "heritage site"
    ],
    'Culture': [
        "culture", "tradition", "festival", "heritage", "art", "music", "dance", "custom", "ceremony",
        "local life", "folklore", "ritual", "cultural", "ethnic", "craft", "handicraft", "language", "performance",
        "theater", "opera", "cultural event", "history", "historical", "museum", "exhibition", "parade",
        "celebration", "religion", "religious", "practice", "community", "local people", "native",
        "ancestry", "origin", "story", "legend", "mythology", "literature", "painting", "sculpture", "friendly"

    ],
    'Cost': [
        "cost", "price", "expensive", "cheap", "affordable", "budget",  "overpriced", "inexpensive",
        "costly", "fee", "charge", "rate", "discount", "deal", "expensiveness", "pricing", "bargain", "money",
        "pay", "payment", "economical", "wallet", "savings", "fare", "bill", "expense",
        "cost-effective", "reasonable", "unreasonable", "rip-off", "overcharge", "undercharge", "promotion"
    ],
    'Accessibility': [
        "access", "accessible", "wheelchair", "stairs", "elevator", "location", "entrance", "exit", "convenient",
        "inconvenient", "easy to reach", "hard to find", "signage", "directions", "navigation", "pathway", "ramps",
        "transport link", "public transport", "parking", "distance", "walk", "walkable", "mobility", "accessibility",
        "handicap", "disabled", "approach", "route", "connection", "transfer", "gate", "door", "corridor", "hallway",
        "lift", "escalator", "stairs", "step", "obstacle", "barrier", "accommodate", "accommodation needs"
    ],
    'Safety': [
        "safe", "safety", "secure", "dangerous", "crime", "police", "theft", "pickpocket", "security", "threat",
        "hazard", "emergency", "accident", "robbery", "danger", "risky", "unsafe", "incident", "protection", "guard",
        "caution", "alert", "violence", "assault", "trustworthy", "reliable", "attack", "harassment", "scam",
        "fraud", "insurance", "lost", "missing", "injury", "medical", "first aid", "ambulance", "fire", "firefighter",
        "rescue", "lifeguard", "patrol",  "supervise", "surveillance"
    ],
    'Accommodation': [
        "hotel", "hostel", "guesthouse", "room", "suite", "villa", "apartment", "bnb", "airbnb", "resort", "lodge",
        "stay", "accommodation", "bed", "pillow", "blanket", "check-in", "check-out", "front desk", "reception",
        "housekeeping", "cleanliness", "comfort", "luxury", "budget stay", "motel", "inn", "homestay", "booking",
        "reservation", "service", "staff", "concierge", "bellboy", "porter", "laundry", "bathroom", "shower",
        "toilet", "balcony", "view", "pool", "spa", "gym", "fitness", "wifi", "internet", "breakfast included",
        "amenities", "facility", "facilities", "mini bar", "fridge", "air conditioning", "heating", "towel", "linen"
    ],
    'Food & Dining': [
        "food", "dining", "restaurant", "cuisine", "meal", "breakfast", "lunch", "dinner", "snack", "dish", "menu",
        "taste", "delicious", "yummy", "flavor", "eat", "eating", "buffet", "cafe", "bar", "beverage", "drink",
        "wine", "coffee", "tea", "dessert", "street food", "local food", "gourmet", "fine dining", "food court",
        "bistro", "chef", "kitchen", "service", "waiter", "waitress", "reservation", "takeaway", "delivery",
        "fast food", "appetizer", "starter", "main course", "entree", "side dish", "supper", "brunch", "tasting",
        "cooking", "culinary", "flavoursome", "spicy", "sweet", "savory", "portion", "serving", "fresh", "organic",
        "vegan", "vegetarian", "gluten free", "allergy", "halal", "kosher"
    ],
    'Transportation': [
        "transport", "bus", "train", "taxi", "car", "rental", "ride", "driver", "uber", "lyft", "shuttle",
        "airport", "flight", "plane", "ferry", "boat", "ship", "tram", "metro", "subway", "bicycle", "bike",
        "motorbike", "scooter", "public transport", "ticket", "fare", "station", "terminal", "commute", "traffic",
        "road", "highway", "parking", "transit", "journey", "trip", "travel", "cab",  "minibus", "van",
        "pickup", "drop off", "arrival", "departure", "route",
         "driver", "pilot", "conductor", "ticket office", "platform",  "bus stop",
        "railway", "express",  "intercity", "international", "boarding"
    ],
    'Shopping & Souvenirs': [
        "shopping", "shop", "market", "mall", "store", "souvenir", "gift", "bazaar", "boutique", "purchase",
        "sale", "discount", "price", "vendor", "stall", "handicraft", "local product", "artisan", "goods", "item",
        "merchandise", "retail", "shopping center", "plaza", "supermarket", "grocery", "department store",
        "window shopping", "bargain", "haggle", "negotiate", "deal", "offer", "promotion", "sale", "clearance",
        "brand", "label", "fashion", "clothing", "jewelry", "accessory", "perfume", "cosmetic", "electronics",
        "appliance", "present", "memento", "keepsake", "collectible", "curio"
    ],
    'Nightlife & Entertainment': [
        "nightlife", "bar", "club", "party", "music", "show", "concert", "pub", "dance", "dj", "karaoke",
        "entertainment", "event", "night", "evening", "performance", "live music", "drink", "cocktail",
        "lounge", "disco", "casino", "theater", "cinema", "movie", "festival", "fireworks", "gig", "comedy",
        "stand-up", "open mic", "cabaret", "burlesque", "strip club", "after dark", "late night", "night out",
        "nightspot", "nightclub", "bouncer", "dance floor", "light show"
    ],
    'Wildlife & Nature': [
        "wildlife", "animal", "nature", "jungle", "forest", "bird", "park", "reserve", "safari", "zoo", "garden",
        "flora", "fauna", "beach", "mountain", "hill", "lake", "river", "waterfall", "scenery", "scenic",
        "natural", "outdoors", "landscape", "tree", "plant", "flower", "ecology", "national park", "marine",
        "reef", "coral", "dolphin", "whale", "turtle", "shark", "elephant", "leopard", "deer", "monkey",
        "insect", "butterfly", "birdwatching", "nature walk", "hiking trail", "cave", "volcano", "island"
    ],
    'Historical & Religious Sites': [
        "temple", "church", "mosque", "shrine", "monastery", "cathedral", "pagoda", "synagogue", "basilica",
        "ruins", "castle", "fort", "palace", "heritage site", "ancient", "historical", "religious", "monument",
        "sanctuary", "chapel", "statue", "archaeological", "museum", "crypt", "tomb", "grave", "cemetery",
        "mausoleum", "relic", "artifact", "pilgrimage", "pilgrim", "sacred", "holy", "worship", "prayer",
        "mass", "sermon", "service", "blessing", "offering", "icon", "fresco", "mural"
    ],
    'Activities & Adventure': [
        "activity", "adventure", "excursion", "tour", "hiking", "surfing", "rafting", "trekking", "diving",
        "snorkeling", "cycling", "biking", "climbing", "paragliding", "zipline", "kayaking", "canoeing",
        "sailing", "camping", "fishing", "golf", "skiing", "snowboarding", "horseback riding", "sports",
        "game", "explore", "exploring", "outing", "trip", "mountaineering", "abseiling", "caving", "spelunking",
        "parachuting", "skydiving", "bungee", "jump", "adrenaline", "extreme", "challenge", "treasure hunt",
        "archery", "paintball", "laser tag", "zip lining", "windsurfing", "paddleboarding", "rock climbing",
        "ice skating", "sledding", "rafting", "canoe", "kayak", "rafting"
    ],
    'Crowds & Over-Tourism': [
        "crowd", "busy", "overcrowded", "tourist", "queue", "noisy", "rush", "jam-packed", "line", "waiting",
        "over-tourism", "packed", "full", "congested", "hustle", "bustle", "touristy", "traffic", "mass tourism",
        "swarm", "mob", "throng", "overrun", "overflow", "jam", "wait time", "delay", "long line", "stand in line"
    ],
       'Sustainability & Eco-Friendliness': [
        "sustainable", "eco-friendly", "green", "environment", "conservation", "recycle",
        "solar", "organic", "responsible", "nature-friendly", "plastic-free", "carbon",
        "renewable", "wildlife protection", "ethical", "local community", "eco lodge",
        "energy saving", "water saving", "compost", "biodegradable", "footprint", "offset",
        "clean energy", "preserve", "biodiversity", "tree planting", "reuse", "upcycle",
        "zero waste", "sustainability", "green initiative", "nature reserve", "garbage", "litter",
        "trash", "waste", "pollution", "rubbish", "plastic", "debris", "dirty", "unclean", "mess", "filthy",
        "contaminated", "waste bin", "waste management", "waste disposal", "illegal dumping", "waste segregation",
        "waste collection", "waste problem", "overflowing bin", "waste site", "waste facility", "waste treatment",
        "recycling", "compost", "environmental cleanliness", "sanitation", "unsanitary", "refuse", "dump", "landfill"
    ],

    'Wellness & Relaxation': [
        "wellness", "relax", "spa", "massage", "manicure", "pedicure", "yoga", "meditation", "calm", "peaceful",
        "serene", "tranquil", "rest", "rejuvenate", "therapy", "treatment", "health", "fitness", "retreat",
        "unwind", "chill", "soothing", "refresh", "detox", "well-being", "pilates", "hot spring", "onsen",
        "jacuzzi", "sauna", "steam room", "aromatherapy", "facial", "body scrub", "relaxation", "mindfulness",
        "stress-free", "balance", "harmony", "zen", "meditate", "healing", "spa treatment", "wellbeing"
    ]
}


Overwriting aspect_vb.py


In [ ]:
import pandas as pd
import re
from tabulate import tabulate
#according to my defined aspects
from aspect_vb import aspect_vb

def label_aspects(sentence, aspect_vb):
    sentence_lower = sentence.lower()
    labels = []
    for aspect, vocab in aspect_vb.items():
        for word in vocab:
            # Check if the aspect word is present as a whole word in the sentence
            if re.search(r'\b' + re.escape(word.lower()) + r'\b', sentence_lower):
                labels.append(aspect)
                break
    return labels if labels else ['Other']

df = pd.read_csv("/content/drive/My Drive/fyp/final-preprocess/Reviews_SriLankan_destinations-sentences.csv")
df = df.head(30)
df['aspect_labels'] = df['review sentences'].apply(lambda x: label_aspects(str(x), aspect_vb))
df['aspect_labels_str'] = df['aspect_labels'].apply(lambda x: ', '.join(x))

df.to_csv("/content/drive/My Drive/fyp/final-preprocess/Reviews_SriLankan_destinations-sentences-aspects.csv", index=False)
print((tabulate(df[['review sentences', 'aspect_labels_str']].head(36), tablefmt="fancy_grid")))

╒════╤═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╤══════════════════════════════════════════════════════════════════════════╕
│  0 │ i had a manicure here and it really was professional and clean                                                                                                                                                                                                                                                    │ Wellness & Relaxation                                                    │
├────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Aspect sentiment label data combibe

In [ ]:
import pandas as pd
from tabulate import tabulate
# Load the two CSV files

df1 = pd.read_csv('/content/drive/My Drive/fyp/final-preprocess/Reviews_SriLankan_destinations-sentences.csv')
df2 = pd.read_csv('/content/drive/My Drive/fyp/final-preprocess/Reviews_SriLankan_destinations_Aspect_Sentiment.csv')
df2['review sentences'] = df2['review sentences'].str.lower()

# Merge on the 'review sentences' column
merged_df = pd.merge(df1, df2, on='review sentences', how='inner')

# Save the merged result
merged_df.to_csv('/content/drive/My Drive/fyp/final-preprocess/Reviews_SriLankan_labeled.csv', index=False)
print(tabulate(merged_df.head(30), headers="keys", tablefmt="fancy_grid"))

╒════╤═════════════════╤════════════════╤═════════════════╤═════════════════════════════╤═══════════════╤══════════════════════╤══════════╤═════════════════╤══════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

In [ ]:
import pandas as pd
from tabulate import tabulate

# Load the merged CSV file
merged_df = pd.read_csv('/content/drive/My Drive/fyp/final-preprocess/Reviews_SriLankan_labeled.csv')

# Ensure 'aspects' column is string type, and strip any whitespace
merged_df['aspects'] = merged_df['aspects'].astype(str).str.strip()

# Remove rows where 'aspects' is exactly '[]'
filtered_df = merged_df[merged_df['aspects'] != '[]']

# Save the filtered dataframe
filtered_df.to_csv('/content/drive/My Drive/fyp/final-preprocess/Reviews_SriLankan_labeled_two.csv', index=False)

# Print the first 30 rows as a table
print(tabulate(filtered_df.head(30), headers='keys', tablefmt='fancy_grid'))

# Print the number of columns
print("\nNumber of raws:", filtered_df.shape[0])


╒════╤═════════════════╤════════════════╤═════════════════╤═════════════════════════════╤═══════════════╤══════════════════════╤══════════╤═════════════════╤══════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

In [ ]:
import pandas as pd
import ast

# Load the dataset
file_path = '/content/drive/My Drive/fyp/final-preprocess/Reviews_SriLankan_labeled_two.csv'
df = pd.read_csv(file_path)

# Parse aspect_sentiments column from string to dictionary
df['aspect_sentiments'] = df['aspect_sentiments'].apply(ast.literal_eval)

# List of specified aspects
aspects = [
    'Attractions', 'Culture', 'Cost', 'Accessibility', 'Safety', 'Accommodation',
    'Food & Dining', 'Transportation', 'Shopping & Souvenirs', 'Nightlife & Entertainment',
    'Wildlife & Nature', 'Historical & Religious Sites', 'Activities & Adventure',
    'Crowds & Over-Tourism', 'Sustainability & Eco-Friendliness', 'Wellness & Relaxation'
]

# Sentiment mapping
sentiment_map = {
    'very negative': 0,
    'negative': 1,
    'neutral': 2,
    'positive': 3,
    'very positive': 4
}

# Create presence and sentiment columns for each specified aspect
for aspect in aspects:
    # 1 if aspect present, else 0
    df[aspect] = df['aspect_sentiments'].apply(lambda d: 1 if aspect in d else 0)
    # Sentiment mapping, -1 if not present
    df[f'Sentiment_{aspect}'] = df['aspect_sentiments'].apply(
        lambda d: sentiment_map.get(d[aspect].lower(), -1) if aspect in d else -1
    )

# Save the updated dataframe to a new CSV
output_path = '/content/drive/My Drive/fyp/final-preprocess/Reviews_SriLankan_destinations_Aspect_Sentiment_expanded_specified.csv'
df.to_csv(output_path, index=False)
# Print the first 30 rows as a table
print(tabulate(df.head(30), headers='keys', tablefmt='fancy_grid'))


╒════╤═════════════════╤════════════════╤═════════════════╤═════════════════════════════╤═══════════════╤══════════════════════╤══════════╤═════════════════╤══════════════════════════════════════════════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════